In [1]:
from copy import copy, deepcopy
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn import metrics
import pandas as pd 
import numpy as np
%matplotlib inline

In [2]:
# Load the Wine Dataset
df_wine = pd.read_csv("data/data_transformed.csv", encoding = 'utf8', index_col=0)
# df_wine = pd.read_csv("../data/winemag-data-130k-v2.csv", encoding = 'utf8', index_col=0)
df_wine = df_wine.reset_index()

In [3]:
df_wine.head(3)

,index,price,points,year_of_wine,v_Abouriou,v_Aglianico,v_Airen,v_Albana,v_Albanello,v_Albariño,...,province_Spanish Islands,province_Tasmania,province_Texas,province_Tuscany,province_Veneto,province_Vermont,province_Victoria,province_Virginia,province_Washington,province_Western Australia
0,2,14.0,87,2012,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,13.0,87,2011,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,65.0,87,2013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Data Preparation

In [4]:
# DEFINE X and Y VARIABLES
df_X = df_wine.drop('points', axis=1)
df_Y = df_wine[['points']]
print(df_X.shape)
print(df_Y.shape)

(76176, 584)
(76176, 1)


# Future Selection & Information Gain

In [5]:
# Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

# Create and fit selector
selector = SelectKBest(f_regression, k = 50)
selector.fit(df_X, df_Y)

mask = selector.get_support()
new_features = df_X.columns[mask]
print('# of cols before:', df_X.shape[1])
df_X = df_X[new_features]
print('# of cols after:', df_X.shape[1])

# of cols before: 584
# of cols after: 50


/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encounter

In [6]:
# INFORMATION GAIN
from sklearn import linear_model

lm = linear_model.LinearRegression()
scores = []

num_features = len(df_X.columns)
for i in range(num_features):
    col = df_X.columns[i]
    scores_col = cross_val_score(lm, df_X[col].values.reshape(-1,1), df_Y.values.ravel(), cv=10, scoring='neg_mean_squared_error')
    best_score = np.mean(np.sqrt(-scores_col))
    scores.append((best_score, col))
    
scores = sorted(scores, reverse = False)
print('top 5 columns:', scores[0:5])

/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


top 5 columns: [(2.7710070616718716, 'price'), (3.0117181569967792, 'v_Pinot Noir'), (3.020919596039174, 'country_Argentina'), (3.0217042143346355, 'country_Spain'), (3.0255993408496704, 'province_Mendoza Province')]


In [7]:
# Split Data to Test and Traning
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33)

# Grid Search For Hyper Parameter Selection

In [8]:
# Best result without Hyperparameter optimisation
lm = linear_model.LinearRegression()

# Train the model using the training sets
lm.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = lm.predict(X_test)

# The mean squared error
print("# Best result WITHOUT hyperparameter optimisation #")
print("RMSE: %.4f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("MAE: %.4f" % metrics.mean_absolute_error(y_test, y_pred))

# Best result WITHOUT hyperparameter optimisation #
RMSE: 2.6403
MAE: 2.1025


In [9]:
# Hyperparameter selection
lm = linear_model.LinearRegression()
parameters = {'fit_intercept':[True,False],
              'normalize':[True,False], 
              'copy_X':[True, False]}

gd_sr = GridSearchCV(estimator = lm,  
                     param_grid = parameters,
                     scoring = "neg_mean_squared_error",
                     cv = 10)

gd_sr.fit(X_train, y_train)  

best_parameters = gd_sr.best_params_  
print("Best Parameters:", best_parameters)

Best Parameters: {'copy_X': True, 'fit_intercept': True, 'normalize': False}


In [10]:
# Best result with Hyperparameter optimisation
lm = linear_model.LinearRegression(copy_X = True, fit_intercept = True, normalize = True)

# Train the model using the training sets
lm.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = lm.predict(X_test)

# The mean squared error
print("# Best result WITH hyperparameter optimisation #")
print("RMSE: %.4f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("MAE: %.4f" % metrics.mean_absolute_error(y_test, y_pred))

# Best result WITH hyperparameter optimisation #
RMSE: 2.6403
MAE: 2.1025
